
# Librerie

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import poppy
import astropy.units as u
import logging
logging.basicConfig(level=logging.DEBUG)
from astropy.io import fits
from astropy.modeling import models, fitting
import tifffile
import numpy as np
import matplotlib.colors as mcolors
from astropy.stats import sigma_clip
logging.getLogger().setLevel(logging.INFO)
from scipy.optimize import curve_fit
import photutils
import time
import datetime as dt
from scipy import fftpack
import nbformat
import plotly.graph_objects as go
from PIL import Image
from turbustat.statistics import PowerSpectrum
from scipy.signal import butter, filtfilt
import ipywidgets as widgets
from IPython.display import display, clear_output
from astropy.io.fits import Header
from scipy.signal import find_peaks
import os
import csv
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus
from photutils.aperture import ApertureStats

# Colormap
dark_yellow = [ '#003f5c', '#2f4b7c', '#665191', '#a05195', '#d45087', '#f95d6a', '#ff7c43', '#ffa600']
custom_cmap = mcolors.LinearSegmentedColormap.from_list("DarkYellow", dark_yellow)
gradient = np.linspace(0, 1, 256).reshape(1, -1)



<span style="color:yellow; font-size:40px;">Variabili in input</span> 


In [ ]:
############### PER SPECKLE FIT ###################################
speckle_threshold=7000 #1100 #7000 #9500 #8000 #9000 #6500 #9000
# Plate scale arcseconds/pixels
plate_scale = 0.0109 # 0.012#4-MARZO #0.0109 #FEBBRAIO# 0.0377 #0.0244*2 #0.0376*2 #0.0268 marzo #0.0244 febb #0.0376 #
# Diffraction limit of the telescope
aperture = 1.82                                                     # Aperture of the telescope (in meters)
wavelength = 633e-9 #700e-9                                                 # Reference wavelenght (in meters) 
diffraction_limit_radians = 2*1.22 * wavelength / aperture          # Diameter of the Airy disk
diffraction_limit_arcseconds = diffraction_limit_radians * 206265
expected_speckle_size_radians = 0.8038 *(1.22 * wavelength / aperture)
expected_speckle_size = expected_speckle_size_radians * 206265  
expected_speckle_size_pixels = expected_speckle_size / plate_scale
# Radius use for the fit of the speckles (in pixels)
radius=int((diffraction_limit_arcseconds/plate_scale)/2)  # raggio della zona del fit attorno alle speckle (Diametro = disco di airy)
check_radius = int((diffraction_limit_arcseconds*0.5)/plate_scale) # raggio della zona di controllo attorno alle speckle (Diametro = disco di airy)
print("Diffraction limit in arcseconds and pixels: ",diffraction_limit_arcseconds, diffraction_limit_arcseconds/plate_scale)
print("Expected speckle size in arcseconds and pixels: ", expected_speckle_size, expected_speckle_size_pixels)

###########    PER PSD   ##################################################
ordine = 5 #  NON OLTRE 6
imagenumber = 300 #500
stacked = False  # Fa PSD di più immagini sommate (3 al momento)
nstack = 3 # immagini da sommare se si vuole usare il Power specrum di immagini sommate
lowcut_pix = 14#15#7  #max dimension in pixel
highcut_pix = 4#9#2.1  #min dimension in pixel
lowcut= 1/lowcut_pix 
highcut = 1/highcut_pix
crop_size = 2000   # FORMATO IMMAGINE FINALE: N pixel X N pixel (dimiuisce la dimensione dell'immagine per velocizzare i calcoli)
frequency_range = 0.15    # range per trovare il picco delle spckle (centrato nella frequenza delle speckle teorica)
########################################################

Diffraction limit in arcseconds and pixels:  0.17504418560439558 16.05909959673354
Expected speckle size in arcseconds and pixels:  0.07035025819440659 6.45415212792721


## <span style="color:green; font-size: 1em;"> Lettura file

In [ ]:

#file = (r"C:\Users\buonc\OneDrive - Alma Mater Studiorum Università di Bologna\Documents\Astrophysics and Cosmology\TESI\dati\Algol_1kHz_1_MMStack_Pos0.ome.tif")
file = (r"C:\Users\buonc\OneDrive - Alma Mater Studiorum Università di Bologna\Documents\Astrophysics and Cosmology\TESI\dati\febbraio\arcturus_633nm_10ms.ome.tif")
#file = (r"C:\Users\buonc\OneDrive - Alma Mater Studiorum Università di Bologna\Documents\Astrophysics and Cosmology\TESI\dati\marzo\4-03\Castor.tif")
#file =(r"C:\Users\buonc\OneDrive - Alma Mater Studiorum Università di Bologna\Documents\Astrophysics and Cosmology\TESI\dati\marzo\4-03\aldebaran_40ms_elev30.tif")
immagini = tifffile.imread(file)[:5000] # LIMITE DI 5000 IMMAGINI PER EVITARE PROBLEMI DI MEMORIA

# <span style="color:orange; font-size: 1em;"> Funzioni

In [ ]:
def calcProcessTime(starttime, cur_iter, max_iter):

    telapsed = time.time() - starttime
    testimated = (telapsed/cur_iter)*(max_iter)
    finishtime = starttime + testimated
    finishtime = dt.datetime.fromtimestamp(finishtime).strftime("%H:%M:%S")
    lefttime = testimated-telapsed 

    return (int(telapsed), int(lefttime), finishtime)
######################################################################################################################
######################################################################################################################

def fit_speckle_tot(data, filtered_speckles, radius, speckle_threshold, plate_scale):
    fwhm_results = []
    centers = []
    
    for speckle in filtered_speckles:
        y_ref, x_ref = speckle  
        masked_data = data.copy() 
        y, x = np.mgrid[y_ref-radius:y_ref+radius+1, x_ref-radius:x_ref+radius+1]
        masked_data = masked_data[y_ref-radius:y_ref+radius+1, x_ref-radius:x_ref+radius+1]
        masked_data[masked_data < 0] = 0 
        
        ###########################################################################
        #Se attivo, fa il fit in una regione circolare
        distance = np.sqrt((x - x_ref)**2 + (y - y_ref)**2)
        circular_mask = distance <= radius
        masked_data = np.where(circular_mask, masked_data, 0)
        ##############################################################################
        
        # Set border pixels to 0
        masked_data[0, :] = 0
        masked_data[-1, :] = 0
        masked_data[:, 0] = 0
        masked_data[:, -1] = 0

        gaussian_model = models.Gaussian2D(amplitude=masked_data.max(), x_mean=x_ref, y_mean=y_ref, x_stddev=0.5, y_stddev=0.5)
        gaussian_model.amplitude.min = speckle_threshold
        gaussian_model.amplitude.max = masked_data.max()
        gaussian_model.x_mean.min = x_ref - 5   # METTERE A 1 PER LE SPECKLE A 3X
        gaussian_model.x_mean.max = x_ref + 5
        gaussian_model.y_mean.min = y_ref - 5
        gaussian_model.y_mean.max = y_ref + 5

        fitter = fitting.LevMarLSQFitter()
        fitted_model = fitter(gaussian_model, x, y, masked_data)

        fwhm_x = 2.355 * fitted_model.x_stddev.value * plate_scale
        fwhm_y = 2.355 * fitted_model.y_stddev.value * plate_scale
        fwhm_results.append((fwhm_y, fwhm_x))
        centers.append((fitted_model.y_mean.value, fitted_model.x_mean.value))

    return np.array(fwhm_results), np.array(centers)
######################################################################################################################
######################################################################################################################

def gaussiana(bins, media, sigma):
	x = np.zeros(len(bins)-1)
	for i in range(len(x)):
		x[i] = (bins[i]+bins[i+1])/2
	y = 1/(sigma*np.sqrt(2*np.pi))*np.exp(-(x-media)**2/(2*sigma**2))
	return x, y
######################################################################################################################
######################################################################################################################

def calculate_2dft(input):
    ft = np.fft.ifftshift(input)
    ft = np.fft.fft2(ft)
    return np.fft.fftshift(ft)

######################################################################################################################
######################################################################################################################

def butterworth_2d_bandpass(image, lowcut, highcut, order=5):
    ny, nx = image.shape
    y, x = np.ogrid[:ny, :nx]
    cy, cx = ny // 2, nx // 2
    # Create normalized radius grid (distance from center)
    radius = np.sqrt((x - cx)**2 + (y - cy)**2)
    radius /= np.max(radius)  # Normalize to [0, 1]
    # Normalize lowcut/highcut to [0, 1] as fraction of Nyquist
    low = lowcut * 2
    high = highcut * 2
    # Butterworth bandpass formula
    def butterworth(freq, cutoff, n):
        return 1 / (1 + (freq / cutoff)**(2 * n))
    # Bandpass = Highpass * Lowpass
    lowpass = butterworth(radius, high, order)
    highpass = 1 - butterworth(radius, low, order)
    bandpass_mask = lowpass * highpass
    # Apply filter in frequency domain
    fft_image = np.fft.fft2(image)
    fft_shifted = np.fft.fftshift(fft_image)
    filtered_fft = fft_shifted * bandpass_mask
    filtered_image = np.fft.ifft2(np.fft.ifftshift(filtered_fft)).real
    return filtered_image, bandpass_mask


######################################################################################################################
######################################################################################################################

def crop_center(image, size):
    """
    Crop a square region of given size around the image center.

    """
    ny, nx = image.shape
    cx, cy = nx // 2, ny // 2
    half = size // 2
    x_min = cx - half
    x_max = cx + half
    y_min = cy - half
    y_max = cy + half
    return image[y_min:y_max, x_min:x_max]

######################################################################################################################
######################################################################################################################

def plot_psd_peak(freqs, ps1D, largest_peak_freq, frequency_range, expected_speckle_size, plate_scale):
    zoom_range = (largest_peak_freq - frequency_range / 2, largest_peak_freq + frequency_range / 2)
    zoom_indices = (freqs >= zoom_range[0]) & (freqs <= zoom_range[1])
    zoom_freqs = freqs[zoom_indices]
    zoom_power = ps1D[zoom_indices]
    plt.figure(figsize=(8, 6))
    plt.loglog(zoom_freqs, zoom_power, label='Zoomed Power Spectrum')  # Emphasize the zoomed region
    plt.axvline(x=largest_peak_freq, color='red', linestyle='--', label=f'Peak: {largest_peak_freq:.4f} pix⁻¹')
    plt.axvline(x=1/(expected_speckle_size/plate_scale), color='magenta', linestyle='--', label='Expected Speckle Size')
    plt.title("Zoomed View of Power Spectrum near Largest Peak")
    plt.xlabel("Spatial Frequency [pix⁻¹]")
    plt.ylabel("Power")
    plt.title("Zoomed View of Power Spectrum near Largest Peak")
    plt.grid(True, which="both", linestyle='--')
    plt.legend()
    plt.show()
    
##################################################################################
######################################################################################
def find_peak_power(freqs, ps1D, expected_speckle_size_pixels, frequency_range):
    """
    frequency_range : Range around the expected speckle frequency to search for the peak.
    largest_peak_freq : Frequency with the largest power within the specified range.
    largest_peak_power : Largest power within the specified frequency range.
    """
    expected_speckle_frequency = 1 / expected_speckle_size_pixels
    lower_freq = expected_speckle_frequency - frequency_range / 2
    upper_freq = expected_speckle_frequency + frequency_range / 2
    #print(expected_speckle_size_pixels,expected_speckle_frequency, lower_freq, upper_freq)
    peak_indices = (freqs >= lower_freq) & (freqs <= upper_freq)

    if not any(peak_indices):
        print("No frequencies selected within the specified range.")
        return None, None

    selected_freqs = freqs[peak_indices]
    selected_power = ps1D[peak_indices]
    largest_peak_index = np.argmax(selected_power)
    largest_peak_freq = selected_freqs[largest_peak_index]
    largest_peak_power = selected_power[largest_peak_index]
    
    return largest_peak_freq, largest_peak_power

# <span style="color:red; font-size: 1em;"> Codice

In [ ]:
import gc

folder_path = r"C:\Users\buonc\OneDrive - Alma Mater Studiorum Università di Bologna\Documents\Astrophysics and Cosmology\TESI\dati\febbraio"
file_list = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.tif')]
#immagini_list = []
for file in file_list: 
    print(f"Found {len(file_list)} files in the folder")
    file_name = os.path.basename(file)
    immagini = tifffile.imread(file)[:5000]
    #immagini_list.append(immagini)
    print(f"Loaded data from {file_name}")
    print(f'Analysing {file_name}...')
    
    ############# SPECKLE FIT ##########################################################
    
    start = time.time()
    fwhm_single_image = []
    fwhm_mean_all = []
    fwhm_median_all = []
    fwhm_std_all = []
    centroid_all = []   # first coordinate in the array is the Y
    rms_all = []        # first coordinate in the array is the Y
    fwhm_mean_filtered_all = []
    fwhm_median_filtered_all = []
    fwhm_std_filtered_all = []
    rms_filtered_all = []
    centroid_filtered_all = []
    ellipticity_mean_all=[]

    for imagenumber in range(len(immagini)):
        data_raw = immagini[imagenumber]
        data_clean = data_raw
        poisson_error = np.sqrt(data_raw)
        background_level = np.median(data_raw)
        background_error = np.sqrt(background_level)
        noise = np.sqrt(np.mean(poisson_error**2) + background_error**2)
        background_estimate = data_raw - noise
        background_estimate[background_estimate < 0] = 0
        data = data_raw - background_level
        data[data < 0] = 0

        ###########################     SOLO PER 10KHZ ################################################
        #center_y, center_x = data.shape[0] // 2, data.shape[1] // 2
        #search_radius = 80  ###per speckle a 3x
        #y, x = np.ogrid[:data.shape[0], :data.shape[1]]
        #mask = (x - center_x)**2 + (y - center_y)**2 <= search_radius**2
        #speckle_coords = np.column_stack(np.where((data > speckle_threshold) & mask))
        ################################################################################################
        # QUANDO NON ATTIVO SPECKLE A 10KHZ
        speckle_coords = np.column_stack(np.where(data > speckle_threshold))
        real_speckles = []

        for coord in speckle_coords:
            y, x = coord
            max_count = data[y, x]
            speckle = coord
            # Check the surrounding pixels within the radius
            for dy in range(-check_radius, check_radius + 1):
                for dx in range(-check_radius, check_radius + 1):
                    ny, nx = y + dy, x + dx
                    if 0 <= ny < data.shape[0] and 0 <= nx < data.shape[1]:
                        if data[ny, nx] > max_count:
                            max_count = data[ny, nx]
                            speckle = [ny, nx]           
            if not any(np.array_equal(speckle, x) for x in real_speckles):
                real_speckles.append(speckle)
                
        real_speckles = np.array(real_speckles)
        filtered_speckles = []
                
        for speckle in real_speckles:
            y, x = speckle
            max_count = data[y, x]
            keep_speckle = True
            # Check for speckles that are closer than N pixels
            for other_speckle in filtered_speckles:
                distance = np.sqrt((other_speckle[0] - y)**2 + (other_speckle[1] - x)**2)
                if distance < 5:   # change here to change the radius (in pixels)
                    if data[other_speckle[0], other_speckle[1]] > max_count:
                        keep_speckle = False
                    else:
                        filtered_speckles = [fs for fs in filtered_speckles if not np.array_equal(fs, other_speckle)]
            if keep_speckle:
                filtered_speckles.append([y, x])

        filtered_speckles = np.array(filtered_speckles)
    ###################################à SOLO PER 10KHZ ##############################################  
        # Calculate the barycenter of the filtered speckles
        #barycenter_x = filtered_speckles[:, 1].mean()
        #barycenter_y = filtered_speckles[:, 0].mean()
    # Define a maximum distance from the barycenter to consider a speckle as valid
        #max_distance = 50  
        #distances = np.sqrt((filtered_speckles[:, 1] - barycenter_x)**2 + (filtered_speckles[:, 0] - barycenter_y)**2)
        #filtered_speckles = filtered_speckles[distances <= max_distance] 
    #############################################################################

       
        fwhm_single_image, centers = fit_speckle_tot(data, filtered_speckles, radius, speckle_threshold, plate_scale)


        ellipticity_single_image = 1 - np.minimum(fwhm_single_image[:, 0], fwhm_single_image[:, 1]) / np.maximum(fwhm_single_image[:, 0], fwhm_single_image[:, 1])

        #tutto con indice 1 perché si sta facendo statistica solo sulla X
        fwhm_x_clipped = sigma_clip(fwhm_single_image[:, 1], sigma=3, maxiters=1)  # o 1 a 3 sigma o 3 a 3.5 sigma
        fwhm_x_tot_clean = fwhm_single_image[~fwhm_x_clipped.mask]
        mean_clipped_fwhm = np.mean(fwhm_x_tot_clean[:,1])
        median_clipped_fwhm = np.ma.median(fwhm_x_tot_clean[:,1])
        std_clipped_fwhm = np.std(fwhm_x_tot_clean[:,1]) 
        centroid = (np.mean(centers[:,0]), np.mean(centers[:,1])) 
        rms = (np.sqrt(np.mean((centers[:,0] - centroid[0])**2)), np.sqrt(np.mean((centers[:,1] - centroid[1])**2))) 
        # mean_clipped_fwhm = np.mean(fwhm_x_tot_clean)
        # median_clipped_fwhm = np.ma.median(fwhm_x_tot_clean)
        # std_clipped_fwhm = np.std(fwhm_x_tot_clean)

        # Store results for all speckles
        fwhm_mean_all.append(mean_clipped_fwhm)
        fwhm_median_all.append(median_clipped_fwhm)
        fwhm_std_all.append(std_clipped_fwhm)
        rms_all.append(rms)
        centroid_all.append(centroid)
        ellipticity_mean_all.append(np.mean(ellipticity_single_image))
        
        if imagenumber % 500 == 0:
            prstime = calcProcessTime(start, imagenumber +1 ,len(immagini))
            print("time elapsed: %s s, time left: %s s, estimated finish time: %s"%prstime)
            
    print('Speckles fit done')
        
############# SALVATAGGIO RISULTATI ############################
    primary_hdu = fits.PrimaryHDU()
    fwhm_mean_hdu = fits.ImageHDU(data=np.array(fwhm_mean_all), name='FWHM_MEAN')
    fwhm_median_hdu = fits.ImageHDU(data=np.array(fwhm_median_all), name='FWHM_MEDIAN')
    fwhm_std_hdu = fits.ImageHDU(data=np.array(fwhm_std_all), name='FWHM_STD')
    centroid_hdu = fits.ImageHDU(data=np.array(centroid_all), name='CENTROID')
    rms_hdu = fits.ImageHDU(data=np.array(rms_all), name='RMS')
    ellipticity_means_hdu = fits.ImageHDU(data=np.array(ellipticity_mean_all), name = 'MEAN ELLIPTICITY')
    hdul = fits.HDUList([primary_hdu, fwhm_mean_hdu, fwhm_median_hdu, fwhm_std_hdu, centroid_hdu, rms_hdu])
    hdul.writeto(f'outputs\results{file_name}.fits', overwrite=True)
    hdul.close()
    print('Speckle fit data saved')

######################## SPECKLE PSD ######################

    print(f'Power Spectrum of{file_name}...')
    
    header = Header()
    header['CDELT1'] = 1.0
    header['CDELT2'] = 1.0
    peak_frequencies = []
    peak_powers = []

    for i, image in enumerate(immagini):
        
        largest_peak_freq = None
        largest_peak_power = None
        
        if stacked and i + nstack <= len(immagini):
            data = np.sum(immagini[i:i + 3], axis=0)
        else:
            data = image
        image_cropped = crop_center(data, crop_size)
        data = image_cropped.copy()
        background_level = np.median(data)
        data = data - background_level
        data[data < 0] = 0
        image = data.copy()

        # Apply 2D Butterworth bandpass
        filtered_image, mask = butterworth_2d_bandpass(image, lowcut, highcut, ordine)
        # Normalize filtered image
        filtered_image = np.nan_to_num(filtered_image, nan=0.0, posinf=0.0, neginf=0.0)
        filtered_image[filtered_image < 0] = 0
        filtered_image /= (np.max(filtered_image) + 1e-8)

        # Compute power spectrum
        pspec_filtered = PowerSpectrum(filtered_image, header=header, distance=1 * u.pc)
        pspec_filtered.run(verbose=False, xunit=u.pix**-1)
        
        freqs = pspec_filtered.freqs.value
        ps1D = pspec_filtered.ps1D
        largest_peak_freq, largest_peak_power = find_peak_power(freqs, ps1D, expected_speckle_size_pixels, frequency_range)

        if largest_peak_freq is not None and largest_peak_power is not None:
            print(f"Largest peak frequency: {largest_peak_freq:.4f} pix⁻¹")
            print(f"Largest peak power: {largest_peak_power:.2f}")
            #print(f"Expected speckle size: {expected_speckle_size:.4f} arcsec")
            #print(f"Expected speckle size in pixels: {expected_speckle_size_pixels:.4f} pixels")
            #plot_psd_peak(freqs, ps1D, largest_peak_freq, frequency_range, expected_speckle_size, plate_scale)
        else:
            print("Could not find any peaks within the specified frequency range.")

        peak_frequencies.append(largest_peak_freq)
        peak_powers.append(largest_peak_power)
        print(f'Power Spectrum of {file_name} done')
        
        ## salvataggio risultati
        output_folder = 'outputs\PSD{file_name}'
        os.makedirs(output_folder, exist_ok=True)
        output_csv_file = os.path.join(output_folder, 'peak_frequencies.csv')
        output_fits_file = os.path.join(output_folder, 'peak_frequencies.fits')
        with open(output_csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Peak Frequencies', 'Peak Power'])
            for freq, power in zip(peak_frequencies, peak_powers):
                writer.writerow([freq, power])
        print(f"Data saved to {output_csv_file}")
        hdu = fits.PrimaryHDU(data=np.array([peak_frequencies, peak_powers]))
        hdul = fits.HDUList([hdu])
        hdul.writeto(output_fits_file, overwrite=True)
        print(f"Data saved to {output_fits_file}")
        
       
        
    
    
    
    del immagini
    gc.collect()
    


Found 1 files in the folder
Loaded data from arcturus_633nm_10ms.ome.tif
Analysing arcturus_633nm_10ms.ome.tif


# <span style="color:blue; font-size: 1em;"> Salvataggio in file

In [ ]:



output_folder = 'outputs'
os.makedirs(output_folder, exist_ok=True)
output_csv_file = os.path.join(output_folder, 'peak_frequencies.csv')
output_fits_file = os.path.join(output_folder, 'peak_frequencies.fits')
with open(output_csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Peak Frequencies', 'Peak Power'])
    for freq, power in zip(peak_frequencies, peak_powers):
        writer.writerow([freq, power])
print(f"Data saved to {output_csv_file}")
hdu = fits.PrimaryHDU(data=np.array([peak_frequencies, peak_powers]))
hdul = fits.HDUList([hdu])
hdul.writeto(output_fits_file, overwrite=True)
print(f"Data saved to {output_fits_file}")